In [657]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
from scipy.ndimage.filters import gaussian_filter
import seaborn as sns
from datetime import datetime

In [658]:
def normalize_df(df):
    df_normalized = df.copy()
    df_normalized['xCoordNorm'] = np.where(df['rinkSide'] == 'left', df['xCoord'], -1 * df['xCoord'])
    df_normalized['yCoordNorm'] = np.where(df['rinkSide'] == 'left', df['yCoord'], -1 * df['yCoord'])

    return df_normalized

In [659]:
def converTimeToHours(t):
    delta_time = datetime.strptime(df['totalPlayTime'][0], '%H:%M:%S') - datetime(1900, 1, 1)
    hours = delta_time.total_seconds() / 3600
    return hours

def totalPlayedTime(df, team):
    df['totalPlayTimeInSeconds'] = df['totalPlayTime'].apply(lambda t: converTimeToHours(t))
    totalTimePerTeam = df.drop_duplicates(subset=['gameID', 'teamOfShooter', 'totalPlayTimeInSeconds'])
    totalTimePerTeam = totalTimePerTeam[['teamOfShooter', 'totalPlayTimeInSeconds']].groupby(['teamOfShooter']).sum()
    hours = totalTimePerTeam.loc[team]['totalPlayTimeInSeconds']
    return hours

In [660]:
def processData(season,team):
    
    # read data     
    df = pd.read_csv('dataset/Allseasons.csv')
        
    # get number of teams    
    nbTeams = len(df['teamOfShooter'].unique())
    
    # normalize coordinates to use half of the rink
    df = normalize_df(df)
    
    # filter events for the given season     
    df=df[df["season"]==season]
    
    # compute number of shots per (x,y) for the league (all the teams)     
    shotsPerXY = df.groupby(['xCoordNorm','yCoordNorm']).size().reset_index(name='nbShotsLeague')
    
    # compute average shots per (x,y) in the league 
    shotsPerXY['avgShotsLeague'] = shotsPerXY['nbShotsLeague'] / nbTeams
    
    # filter shots for the given team
    teamShots = df.loc[df['teamOfShooter'] == team]
    
    # compute number of shots for the given team per (x,y)     
    teamShotsPerXY = teamShots.groupby(['xCoordNorm','yCoordNorm']).size().reset_index(name='nbShotsTeam')
    
    # Compute excess shots: excess shots = nb shots for a given team - average league shots
    excessShots = pd.merge(teamShotsPerXY, shotsPerXY, on=['xCoordNorm', 'yCoordNorm']) 
    excessShots['excessShots'] = excessShots['nbShotsTeam'] - excessShots['avgShotsLeague']
    
    # Compute played time by 'team' in hours
    hours = totalPlayedTime(df, team)
    
    excessShots['excessShotsPerHour'] = excessShots['excessShots'] / hours

    return excessShots

In [661]:
def getTeams():
    df = pd.read_csv('dataset/Allseasons.csv')
    return df["teamOfShooter"].unique()

In [676]:
def myplot(x, y, z, s, bins=1000):
    heatmap, xedges, yedges = np.histogram2d(x, y, bins=bins, weights=z)
    heatmap = gaussian_filter(heatmap, sigma=s)

    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    return heatmap.T, extent

def plotShotMap(season_year, team, season_games):
    fig, ax = plt.subplots(figsize=(13,8.5))

    img = plt.imread('../figures/nhl_rink.png')

    ext = [-100, 100, -42.5, 42.5]
    plt.imshow(img, zorder=0, extent=ext, aspect=1)
    # Generate some test data
    x = season_games['xCoordNorm']
    y = season_games['yCoordNorm']
    z = season_games['excessShotsPerHour']
    
    sigma = 2
    
    img, extent = myplot(x, y, z, sigma, bins=100)
    img = ax.imshow(img, extent=[0, 100, -42.5, 42.5], origin='lower', cmap='seismic', alpha=.3)#, clim=(-1, 1))
    
    plt.colorbar(img, ax=ax)
    ax.set_title(f"Shot map of {team} - season {season_year}/{season_year+1}")
    
    plt.show()
    
    # Export the figure to an HTML format
    fig.write_html("exports/interactiveShotMap.html")


In [677]:
@interact(
    season_year=widgets.IntSlider(min=2016, max=2020, step=1),
    team=getTeams()
)
def show_data(season_year, team):
    season_games = processData(season_year, team)
    plotShotMap(season_year, team, season_games)
    display(season_games.head())

interactive(children=(IntSlider(value=2016, description='season_year', max=2020, min=2016), Dropdown(descripti…